In [ ]:
import sys
import jsonlines
import os
import pprint
import time
import random
import re
import json
import math

pp = pprint.PrettyPrinter()
sys.path.insert(0, "e:\\Documents\\NLP\\FEVER2021_SharedTask\\FEVEROUS\\src")

DIR_PATH = "e:\\Documents\\NLP\\FEVER2021_SharedTask\\"
TRAIN_DATA_PATH = os.path.join(DIR_PATH, 'data\\train.jsonl')

from database.feverous_db import FeverousDB
from utils.wiki_page import WikiPage

### Import data

In [ ]:
db = FeverousDB("C:/Databases/feverous_wikiv1.db")

In [ ]:
def replace_entities(sent):
    regex = r'\[\[([^\|]+)\|([^\]]+)\]\]'
    return re.sub(regex, '\\2', sent)
  
def remove_punctuation(sent):
    if sent[-1] == '.':
        return sent[:-1]
    else:
        return sent

def extract_sents(doc_json):
    page = WikiPage(doc_json['title'], doc_json)
    sents = [replace_entities(sent.content) for sent in page.get_sentences()]
    sents = [sent.lower() for sent in sents]
    # sents = [remove_punctuation(sent) for sent in sents]
    return sents

def create_sample_docs(ids):
    json_docs = db.get_doc_jsons(ids)
    curr_sample_docs = dict()
    for doc in json_docs:
        sents = extract_sents(doc)
        doc_text = ' '.join(sents)
        curr_sample_docs[doc['title']] = doc_text
    return curr_sample_docs

def write_corpora_to_file(i, docs):
    start_time = time.time()
    FILE_PATH = 'data\\corpora_{}.json'.format(i)
    with open(DIR_PATH + FILE_PATH, 'w') as f:
        f.write(json.dumps(docs, indent=2))
    print("Wrote {} docs to {}: {} seconds".format(SAMPLE_SIZE, FILE_PATH, time.time()-start_time))    

### Create sample docs using multiple threads

In [ ]:
import concurrent.futures
import time


NR_OF_THREADS = 1

with concurrent.futures.ThreadPoolExecutor() as executor:
    thread_samples = int(SAMPLE_SIZE / NR_OF_THREADS)
    start_time = time.time()
    futures = []
    for i in range(NR_OF_THREADS):
        start = thread_samples*i
        ids = sample_doc_ids[start:start+thread_samples]
        futures.append(executor.submit(create_sample_docs, ids))

sample_docs = dict()

for f in futures:
    sample_docs.update(f.result())

print("Creating {} sample docs with {} threads: {} seconds".format(SAMPLE_SIZE, NR_OF_THREADS, time.time()-start_time))    

### Create sample docs on a single thread

In [ ]:
start_time = time.time()
sample_docs = create_sample_docs(sample_doc_ids)
print("Creating {} sample docs: {} seconds".format(SAMPLE_SIZE, time.time()-start_time))    

### Create corpora docs for full dataset

In [ ]:
import math

SAMPLE_SIZE = 100000
NR_OF_DOCS = len(doc_ids)
iterations = int(math.ceil(NR_OF_DOCS/SAMPLE_SIZE))

for i in range(iterations):
    start = i*SAMPLE_SIZE
    end = (i+1)*SAMPLE_SIZE
    if end > NR_OF_DOCS:
        end = NR_OF_DOCS
    print("Creating docs for samples {} to {}".format(start, end))    
    start_time = time.time()    
    ids = doc_ids[start:end]
    docs = create_sample_docs(ids)
    print("Creating {} docs took: {}".format(SAMPLE_SIZE, time.time()-start_time))
    write_corpora_to_file(i+1, docs)

print("Finished creating corpora files!")